<a href="https://colab.research.google.com/github/ANadalCardenas/attention/blob/main/%5BAIDL_ONL_2025_26%5D_L01_Preprocessing_(todo).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sentiment Analysis: Data preprocessing

In this notebook we are going to compare different approaches of data preprocessing applied to a real world task, analysing movie reviews given by IMBD users. Each review can be classified in two different classes, positive, if the user likes the movie and negative otherwise. This tutorial is inspired by: https://towardsdatascience.com/sentiment-analysis-with-python-part-1-5ce197074184

## Data preprocessing

Our first step will be to load and prepare all the our data to perform the experiments. In this case we are going to employ IMBD reviews data available at The Training Dataset used is stored in the zipped folder: aclImbdb.tar file. This can also be downloaded from: http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
The dataset consists in 50.000 sentences splitted in two sets, train and test of 25.000 sentences each.

For our experiments the corpus will be splitted in the following way:
* Train_split: 17500 sentences extracted from the original training data.
* Validation split: 7500 sentences extracted from the original training data.
* Test split: 25000 sentences that form the original test data.

In [1]:
!pip install torchdata
!pip install 'portalocker>=2.0.0'
!pip install datasets
import nltk
nltk.download('omw-1.4')
nltk.download('punkt_tab')

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [2]:
import torch
from datasets import load_dataset

# Load dataset
dataset = load_dataset("imdb")
import random
import itertools
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split


SEED = 0

train_data = dataset["train"]
test_data = dataset["test"]
train_split = list(train_data)
test_split = list(test_data)
train_sents = [t["text"].split() for t in train_split]
test_sents = [t["text"].split() for t in test_split]

train_targets = [t["label"] for t in train_split]
test_targets = [t["label"] for t in test_split]


train_sents, valid_sents, train_targets, valid_targets = train_test_split(train_sents,
                                                                          train_targets,
                                                                          test_size=0.25,
                                                                          random_state=SEED)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

plain_text/train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

plain_text/test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

plain_text/unsupervised-00000-of-00001.p(…):   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In order to feed our data into a model that is able to predict the sentiment of our movie reviews we should first create a vocabulary of all the words that appear in our training data.

A measure of quality of our vocabulary can be its coverage over the data. We can define the vocabulary coverage as the percentage of tokens from our data that are found in our vocabulary.

As we will see later, vocabulary size is a paremeter that can be tuned and this coverage can serve as guidande.

**Compute the vocabulary as a dictionary wirh words as keys and the number of times the word appears as value.**

**Compute the coverage over the data given our vocabulary.**

In [3]:
def compute_vocabulary(train_sents):
  vocabulary = {'<unk>':99999999}
  for sent in train_sents:
    for word in sent:
      if word in vocabulary:
        vocabulary.update({word:vocabulary[word]+1})
      else:
        vocabulary[word] = 1

  return vocabulary

def coverage(split,voc):
  total_words = 0
  words_in_voc = 0
  for sent in split:
    for word in sent:
      total_words += 1
      if word in voc:
        words_in_voc += 1
  if total_words == 0:
    return 0.0
  return (words_in_voc / total_words) * 100

def voc_stats(split,voc):
  print('**** VOCABULARY ***')
  print('* Unique words', len(voc))
  print('* Coverage', coverage(split,voc))

voc = compute_vocabulary(train_sents)
voc_stats(valid_sents, voc)

**** VOCABULARY ***
* Unique words 234010
* Coverage 96.5128798732154


Our first proposed preprocess is word level tokenization. In this cases all word will be splitted and stop-words will be separated. In this case we are going to employ 'nltk' library for this task.

In [4]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import TweetTokenizer
import numpy as np


stop_words = list(stopwords.words('english')) #About 150 stopwords
lemmatizer = WordNetLemmatizer()

def tokenize_sentence(sentence):
    return  nltk.word_tokenize(sentence)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


**Compute the tokenization and vocabulary given the function avobe. How the vocabulary size and the coverage have changed?**

In [11]:
train_tok_sents = [tokenize_sentence(' '.join(s)) for s in train_sents]
valid_tok_sents = [tokenize_sentence(' '.join(s)) for s in valid_sents]
test_tok_sents = [tokenize_sentence(' '.join(s)) for s in test_sents]


In addition to the previous steps we are going further preprocess our data.

For each review in the corpus we apply a preprocess consisting in:

- Each word is replaced by its lemma form. Lemmatization reduces vocabulary size as all different forms of a word are grouped in a single lemma.
- Remove casing from words, this helps reduce ambiguity as upper and lower cased appearences of a word are selected as different in the vocabulary.
- Remove stop words. In order to reduce sentence length and represent the words that carry the meaning of the sentence, we remove all stop words.

**Apply the preprocess and compute its new vocabulary. How did stats change?**


In [13]:
def preprocess_sentence(sentence):
    return  [lemmatizer.lemmatize(word.lower()) for word in sentence if not word in stop_words]


In [14]:
train_tok_sents = [preprocess_sentence(s) for s in train_tok_sents]
valid_tok_sents =  [preprocess_sentence(s) for s in valid_tok_sents]
test_tok_sents =  [preprocess_sentence(s) for s in test_tok_sents]

tok_voc = compute_vocabulary(train_tok_sents)
voc_stats(valid_tok_sents,tok_voc)

**** VOCABULARY ***
* Unique words 89198
* Coverage 98.50315916593148


In both cases vocabularies are too big to be handled in this task as a lot of words only appear a few times in the dataset of some ambiguity is still present in the data.

**Create a 5000 token vocabulary that only include the most frequent tokens in the dataset. How is the new coverage?**

In [16]:
def reduce_vocabulary(voc,size=5000):
  sorted_voc = dict(sorted(voc.items(), key=lambda item: item[1], reverse=True))
  return dict(itertools.islice(sorted_voc.items(), size))

SIZE_5000 = 5000
tok_voc = reduce_vocabulary(tok_voc,size=SIZE_5000)
voc = reduce_vocabulary(voc,size=SIZE_5000)
voc_stats(valid_tok_sents,tok_voc)

**** VOCABULARY ***
* Unique words 5000
* Coverage 87.82846712153088


The previous preprocess helps to reduce the ambiguity produced by the different
form of the same word or stop-words included in other tokens, but the original tokens are mostly unchanged. In the following cases we will explore other levels of tokenization, characters and subwords.

Let's start with characters, where all words in the dataset are stripped into its individual characters, which has several new characterisitics:
* Vocabularies are orders of magnitude smaller that its word counterparts, as all words share the same script that uses a limited set of them.
* Each token does not provide a lot of information about the sentence. Individual words include semantic and morpghological information.
* Character's use is more ambiguious, words are used generally on the same context consistently during the dataset while characters can belong to a great nunmber of words.

**Based on the previous tokenization compute a character vocabulary over the training, and measure its size and coverage of the validation data**

In [20]:

train_char_sents = [ch for sent in train_sents for word in sent for ch in word]
train_char_sents =  [ch for sent in valid_sents for word in sent for ch in word]
valid_char_sents =  [ch for sent in test_sents for word in sent for ch in word]

char_voc =  compute_vocabulary(train_char_sents)
voc_stats(valid_char_sents,char_voc)

**** VOCABULARY ***
* Unique words 149
* Coverage 99.99939976130008


Character level may be too extreme for some tasks, but it provides a great coverage of the dataset. In those cases a great alternative e to apply is subword tokenization.

In this case words are splitted in pieces which lenght depends on how common they are in the dataset. This way long pieces that are really common will be maintained, for example lemmas of common words in the data, while for not common words or affixes the vocabulary includes smaller pieces until arriving to  individual characters.

This tokenizations allows:
* A great coverage of the data, as only words including characters not present in the training data will not be recognized.
* A parametrized vocabulary size. The number of tokens in the vocabulary is set when the tokenization is computed and can be tuned to improve the models performance.

The example shows a call to Byte-Pair-Encoding tokenization using the standard subword-nmt library.

In [22]:
! pip install subword-nmt

# Write the data splits into files to compute the Byte-Pair Encoding (BPE)
with open('train.txt','w+') as tr:
  for s in train_tok_sents:
    print(' '.join(s),file=tr)

with open('valid.txt','w+') as vl:
  for s in valid_tok_sents:
    print(' '.join(s),file=vl)

with open('test.txt','w+') as ts:
  for s in test_tok_sents:
    print(' '.join(s),file=ts)


# Compute BPE codes and apply to the data splits
! subword-nmt learn-bpe -s 5000 < train.txt > bpe5000.codes
! subword-nmt apply-bpe -c bpe5000.codes < train.txt > train.bpe.txt
! subword-nmt apply-bpe -c bpe5000.codes < valid.txt > valid.bpe.txt
! subword-nmt apply-bpe -c bpe5000.codes < test.txt > test.bpe.txt

train_bpe_sents = []
with open('train.bpe.txt') as tr:
  for line in tr.readlines():
    train_bpe_sents.append(line.replace('\n','').split())

  valid_bpe_sents = []
with open('valid.bpe.txt') as tr:
  for line in tr.readlines():
    valid_bpe_sents.append(line.replace('\n','').split())

test_bpe_sents = []
with open('test.bpe.txt') as tr:
  for line in tr.readlines():
    test_bpe_sents.append(line.replace('\n','').split())


100% 5000/5000 [00:41<00:00, 121.94it/s]


**Compute the coverage and vocabulary size of the subword tokenization**

In [ ]:
bpe_voc = compute_vocabulary(train_bpe_sents)
voc_stats(valid_bpe_sents,bpe_voc)

**Pretrained Tokenizers from Language Models:**

Pretrained tokenizers are components of large, pretrained language models (like BERT, GPT, or RoBERTa). These tokenizers come with a pre-built vocabulary and a set of rules for splitting text into tokens (e.g., words or subwords). They have been trained on massive amounts of text data and are designed to work seamlessly with their corresponding language models.

Using a pretrained tokenizer has two main advantages:


*   Consistency: The tokenizer’s vocabulary and rules match those used during the

*   model's training, ensuring that your text is processed in the same way.
Convenience: Instead of building your own tokenizer and managing a vocabulary manually, you can simply load an existing one using libraries such as Hugging Face Transformers.



In [ ]:
from transformers import AutoTokenizer

def tokenize_with_pretrained(model_name, sentences):
    # Load the tokenizer corresponding to the given model name.
    # This retrieves a pretrained tokenizer with its associated vocabulary and tokenization rules.
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    # Check if the sentences are provided as lists of tokens.
    # If so, join them into raw text strings (because the tokenizer expects plain text).
    if isinstance(sentences[0], list):
        sentences = [" ".join(s) for s in sentences]

    # Tokenize the sentences:
    # - 'padding=True' ensures all sequences are padded to the same length.
    # - 'truncation=True' limits sequences that are too long.
    # - 'max_length=512' sets the maximum allowed length of each tokenized sentence.
    encoded = tokenizer(sentences,
                        padding=True,
                        truncation=True,
                        max_length=512)

    # Convert the token IDs back into token strings.
    # This returns a list where each element corresponds to the tokenized representation of a sentence.
    return [tokenizer.convert_ids_to_tokens(ids) for ids in encoded["input_ids"]]


**Tokenize the sentences with the BERT tokenizer**


In [ ]:
# Tokenize with BERT-base-uncased and XLM-RoBERTa-base
bert_tokenized_train = tokenize_with_pretrained("bert-base-uncased", train_sents)
# xlmr_tokenized_train = tokenize_with_pretrained("xlm-roberta-base", train_tok_sents)

bert_tokenized_valid = tokenize_with_pretrained("bert-base-uncased", valid_sents)
# xlmr_tokenized_valid = tokenize_with_pretrained("xlm-roberta-base", valid_tok_sents)

bert_tokenized_test = tokenize_with_pretrained("bert-base-uncased", test_sents)
# xlmr_tokenized_test = tokenize_with_pretrained("xlm-roberta-base", test_tok_sents)

In [ ]:
bert_voc = compute_vocabulary(bert_tokenized_train)
voc_stats(bert_tokenized_valid,bert_voc)

# Classification Task

In this final section we are going to measure the performance of a model when using as features the different tokenization levels described above.

The first step in order to train a model is decide how to feed our data into the model. For these experiments, we are going to employ bag of words vectors as they do not require any further preprocess.

Bag of Words (BOW) consists in representing each of the sentences of the dataset as fixed size vectors of the size of the vocabulary. For example, for our 5000 tokens vocabularies, each sentence will be represented as a 5000 dimension vectors, following these steps:

- Initially all vector dimensions are initialized as 0.
- For each word in the sentence, 1 is added to the position of such word in the vocabulary.

The final vector is a sparce vector, most of its values are 0, that contains the number of times each word appears in the sentence and the sum of all dimensions of the vector is the number of tokens in the sentence.

Note, that while this representation provides a fixed size representation of the sentences it lacks information of the order in which words appear.

**Create a general model to compute the vocabulary indexes and the BOW representations of our data**

In [ ]:
def idx_voc(voc):
  items = list(voc.items())
  items.sort(key=lambda x: x[1], reverse=True)
  return {item[0]:n for n,item in enumerate(items)}

def bag_of_words(splits,voc):
  voc = idx_voc(voc)
  bow_splits = []
  # TODO
  return bow_splits



As a classifier we are going to employ Random forest (https://towardsdatascience.com/understanding-random-forest-58381e0602d2), a model that consists in an ensemble of Decision Trees that see different data by means of bagging and boosting.


When you use a pretrained BERT tokenizer, it splits text into subword units rather than whole words. Applying a simple bag-of-words model on these subwords can lead to an extremely large and fragmented vocabulary, resulting in very high-dimensional and sparse representations. This dense representation, if not handled correctly, can be both computationally and memory inefficient. In contrast, TF-IDF creates a sparse matrix that only stores non-zero entries and also weights tokens by their importance across the corpus. This sparse format is more memory-friendly and better suited for handling the many subword tokens produced by BERT tokenization.

**TF-IDF Vectorization:**
The TfidfVectorizer is used to convert the text into a numerical representation.
When fitting on the training texts with vectorizer.fit_transform(train_texts), the vectorizer learns the vocabulary and calculates the inverse document frequency (IDF) for each token.
The test texts are then transformed using vectorizer.transform(test_texts) based on the vocabulary learned from the training data.
The parameter max_features=5000 limits the number of features (tokens) to 5000 to control memory usage and potentially reduce noise.


*   For character-level processing:
Each sentence (which is a list of characters) is joined using "".join(sent) to create a continuous string without spaces. This ensures that each character is preserved in sequence.

*   For word-level processing:
Each sentence (a list of word tokens) is joined with spaces using " ".join(sent). This creates a typical sentence string where words are separated by spaces, which is what the vectorizer expects.



In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

def compute_tfidf(splits, char_level=False):
    """
    Computes TF-IDF features for training and testing datasets.

    Parameters:
        splits (list): A list of three elements where:
                        - splits[0] is the training data,
                        - splits[1] is the validation data (unused here),
                        - splits[2] is the test data.
                        Each element should be a list of sentences, and each sentence is a list of tokens.
        char_level (bool): If True, use character-level tokenization; otherwise, use word-level tokenization.

    Returns:
        train_tfidf, test_tfidf: The TF-IDF transformed training and test data.
    """
    if char_level:
        # If char_level is True, we configure the vectorizer to break text into characters.
        # analyzer='char' means that individual characters (or character n-grams) are the tokens.
        # ngram_range=(1, 1) specifies that we only consider individual characters.
        # max_features limits the vocabulary size to 5000 to avoid memory issues.
        vectorizer = TfidfVectorizer(analyzer='char', ngram_range=(1, 1), max_features=5000)
        # For character-level analysis, join each sentence's characters without spaces.
        # This produces a continuous string where each character is kept in order.
        train_texts = ["".join(sent) for sent in splits[0]]
        test_texts = ["".join(sent) for sent in splits[2]]
    else:
        # Default behavior uses word-level tokenization.
        # The vectorizer will automatically tokenize the text into words.
        vectorizer = TfidfVectorizer(max_features=5000)
        # For word-level analysis, join tokens with a space to form a proper sentence.
        train_texts = [" ".join(sent) for sent in splits[0]]
        test_texts = [" ".join(sent) for sent in splits[2]]

    # Fit the vectorizer on the training data to learn the vocabulary and IDF values,
    # then transform the training texts into TF-IDF feature matrices.
    train_tfidf = vectorizer.fit_transform(train_texts)
    # Use the learned vocabulary to transform the test texts into TF-IDF feature matrices.
    test_tfidf = vectorizer.transform(test_texts)

    return train_tfidf, test_tfidf


As a classifier we are going to employ Random forest (https://towardsdatascience.com/understanding-random-forest-58381e0602d2), a model that consists in an ensemble of Decision Trees that see different data by means of bagging and boosting.

In [ ]:
from sklearn.ensemble import RandomForestClassifier


def get_accuracy(preds,labels):
    return sum([p == l for p,l in zip(preds,labels)])/len(preds)*100

def trainRandomForestClassifier(train_idx,test_idx):
  train_idx = np.array(train_idx,dtype='float')
  test_idx = np.array(test_idx,dtype='float')

  clf = RandomForestClassifier(n_estimators=10, max_depth=3,random_state=SEED)
  clf.fit(train_idx,train_targets)

  preds = clf.predict(test_idx)
  print('Accuracy', get_accuracy(preds,test_targets),'%')

**Train the model for the different preprocesses. How they affect the results?**

In [ ]:
# Tokenized at word level and preprocessed data
train_idx,valid_idx,test_idx = bag_of_words([train_tok_sents,valid_tok_sents,test_tok_sents],tok_voc)
trainRandomForestClassifier(train_idx,test_idx)


In [ ]:
#Tokenized and character data
train_idx,valid_idx,test_idx = bag_of_words([train_char_sents,valid_char_sents,test_char_sents],char_voc)
trainRandomForestClassifier(train_idx,test_idx)

In [ ]:
#BPE tokenized data
train_idx,valid_idx,test_idx = bag_of_words([train_bpe_sents,valid_bpe_sents,test_bpe_sents],bpe_voc)
trainRandomForestClassifier(train_idx,test_idx)

In [ ]:
# NOT tokenized data
train_idx,valid_idx,test_idx = bag_of_words([train_sents,valid_sents,test_sents],voc)
trainRandomForestClassifier(train_idx,test_idx)

**Running the same experiment with TF-IDF and comparing the results**



In [ ]:
train_idx, test_idx = compute_tfidf([train_tok_sents, valid_tok_sents, test_tok_sents])
trainRandomForestClassifier(train_idx,test_idx)

In [ ]:
train_idx, test_idx = compute_tfidf([train_char_sents,valid_char_sents,test_char_sents],True)
trainRandomForestClassifier(train_idx,test_idx)

In [ ]:
train_idx, test_idx = compute_tfidf([train_bpe_sents,valid_bpe_sents,test_bpe_sents])
trainRandomForestClassifier(train_idx,test_idx)

In [ ]:
train_idx, test_idx = compute_tfidf([train_sents,valid_sents,test_sents])
trainRandomForestClassifier(train_idx,test_idx)

**Now we are going to try the BERT tokenizer with TF-IDF**

---



In [ ]:
train_idx, test_idx = compute_tfidf([bert_tokenized_train, bert_tokenized_valid, bert_tokenized_test])
trainRandomForestClassifier(train_idx,test_idx)

# Conclusions
* Preprocessing plays an important role in the performance of NLP systems.
* Even for the same model performance may vary a lot depending on how expressive are the features selected for the task.
* All methods  presented (with the exception of no preprocess) are employed in state of the art work for several tasks.
